# Water Potability

## Introduction

An important aspect of life is access to safe drinking water. In many places around the world, people do not have easy access, or sometimes any access at all, to clean drinking water. According to the World Health Organization, "over 2 billion people live in water-stressed countries" and "use a drinking water source contaminated with faeces" (https://www.who.int/news-room/fact-sheets/detail/drinking-water). With such a pressing issue, the ability to identify Potable Water accurately and effectively is of utmost importance.

## Applications

Our group aims to use the chemical properties of water to identity whether the body of water is potable. Because these chemical properties can be identified using technology, a device could be constructed to automatically and periodically test a body of water and then use our model to determine whether the water is safe to drink. This device could track the potability of water and notify officials of a change in status, reducing the change of adverse health affects of unsafe drinking water.

## The Dataset

Our dataset contains properties of 3,276 different bodies of water. Each property is a metric of the quality of the water; however, no one property can be used in isolation. The potability of the body of water is determined by the combination of its many attributes.

**Link to Dataset: https://www.kaggle.com/datasets/adityakadiwal/water-potability**

### Description of Attributes

(These are modified from the descriptions from https://www.kaggle.com/datasets/adityakadiwal/water-potability)

**pH value**

PH is an important parameter in evaluating the acid–base balance of water. It is also the indicator of acidic or alkaline condition of water status.

**Hardness**

Hardness is mainly caused by calcium and magnesium salts. These salts are dissolved from geologic deposits through which water travels. The length of time water is in contact with hardness producing material helps determine how much hardness there is in raw water. Hardness was originally defined as the capacity of water to precipitate soap caused by Calcium and Magnesium.

**Solids (Total Dissolved Solids - TDS)**

Water has the ability to dissolve a wide range of inorganic and some organic minerals or salts such as potassium, calcium, sodium, bicarbonates, chlorides, magnesium, sulfates etc. These minerals produced un-wanted taste and diluted color in appearance of water.

**Chloramines**

Chlorine and chloramine are the major disinfectants used in public water systems. Chloramines are most commonly formed when ammonia is added to chlorine to treat drinking water.

**Sulfate**

Sulfates are naturally occurring substances that are found in minerals, soil, and rocks. They are present in ambient air, groundwater, plants, and food.

**Conductivity**

Pure water is not a good conductor of electric current rather’s a good insulator. Increase in ions concentration enhances the electrical conductivity of water. Generally, the amount of dissolved solids in water determines the electrical conductivity.

**Organic Carbon**

Total Organic Carbon (TOC) in source waters comes from decaying natural organic matter (NOM) as well as synthetic sources. TOC is a measure of the total amount of carbon in organic compounds in pure water.

**Trihalomethanes**

THMs are chemicals which may be found in water treated with chlorine.

**Turbidity**

The turbidity of water depends on the quantity of solid matter present in the suspended state. It is a measure of light emitting properties of water and the test is used to indicate the quality of waste discharge with respect to colloidal matter.